In [1]:
from queryAnalyzer import getPlayersInQuery, getLeaguesInQuery, getTeamsInQuery, getSeasonsInQuery, getStagesInQuery
import sqlAnswers

In [2]:
import sqlite3
sqlite_file = 'database.sqlite'

query1 = "Thomas Mueller ist der beste Spieler der Welt!"
query1 = "Müller Thomas ist der beste Spieler der Welt, nicht Aaron Doran!"
query2 = "Bundesliga und Jupiler League sind supi!"
query2 = "Die deutsche Liga ist supi!"


In [3]:
#make query insensitive for umlaute
def preProcessQuery(query):
    umlaut_a = 'ä'.encode()
    umlaut_A = 'Ä'.encode()
    umlaut_o = 'ö'.encode()
    umlaut_O = 'Ö'.encode()
    umlaut_u = 'ü'.encode()
    umlaut_U = 'Ü'.encode()
    
    query = query.encode()
    
    query = query.replace(umlaut_a, b'ae')
    query = query.replace(umlaut_A, b'Ae')
    query = query.replace(umlaut_o, b'oe')
    query = query.replace(umlaut_O, b'Oe')
    query = query.replace(umlaut_u, b'ue')
    query = query.replace(umlaut_U, b'Ue')
    
    query = query.decode('utf-8')
    
    if(query[-1:] == '?' or query[-1:] == '!' or query[-1:] == '.' ):
        return query[:-1]
    return query

Query wird verarbeitet:
    
- Liefert eine playerID eines Spielers zurück
- Keyword "wiegt" (oder auch "schwer") wird gefunden

-> dieses keyword hat nen pointer auf eine SQL Query mit Platzhaltern, in welches die ID des Spielers von dem gesetzt wird


'SELECT weight FROM player WHERE id = playerID'


flair tool 
was passiert wenn der Input da rein kommt

Queries mit "Tags"

Spieler
Ligen
Verein (evtl abkürzung)
Saison / Jahr





In [4]:
def getTeamMatches(): #testausrufe
    import sqlite3
    conn = sqlite3.connect('database.sqlite')
    c = conn.cursor()
    queryText = 'SELECT * FROM match WHERE id = 307'
    c.execute(queryText)
    result = c.fetchall()
    conn.close()
    return result
print(getTeamMatches())

[(307, 1, 1, '2009/2010', 1, '2009-07-31 00:00:00', 665318, 9985, 9997, 2, 2, 1, 2, 4, 6, 8, 2, 4, 6, 8, 4, 6, 1, 2, 4, 6, 8, 2, 4, 6, 8, 4, 6, 1, 3, 3, 3, 3, 7, 7, 7, 7, 10, 10, 1, 3, 3, 3, 3, 7, 7, 7, 7, 10, 10, 38797, 39580, 38800, 37861, 47411, 35412, 39631, 39591, 37262, 25957, 38369, 37868, 33595, 38347, 3329, 149150, 38354, 37866, 36836, None, 5016, 45490, None, None, None, None, None, None, None, None, 1.22, 6, 13, 1.2, 5.5, 10.5, 1.2, 5, 10, 1.17, 5.5, 12, None, None, None, 1.2, 5, 11, 1.18, 6.5, 13, 1.2, 5.5, 12, 1.2, 5.5, 11, 1.17, 6, 11)]


In [5]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings

# create a StackedEmbedding object that combines crawl and forward/backward flair embeddings 
#(which take the words next to it into account before giving a vector)
german_embedding = StackedEmbeddings([
                                        WordEmbeddings('de-crawl'),
                                        FlairEmbeddings('de-forward'),
                                        FlairEmbeddings('de-backward'),
                                       ])
#needed to check for unknown (e.g. non german) or wrong spelled words
crawl_embedding = WordEmbeddings('de-crawl')
from flair.data import Sentence
 
import numpy
import json

In [8]:
#returns the nearest vector from the query to the individual training sentences
def getVectorOfAnswer(index, queryV):
    with open('trainData.json', 'r') as myfile:
        data=myfile.read()
    obj = json.loads(data)
    bestEmbedding = 0
    inIteration1 = True
    for sentence in obj[str(index)]:
        
        sentence = Sentence(preProcessQuery(sentence))
        german_embedding.embed(sentence)
        
        embed = 0
        for token in sentence:
            
            embed += token.embedding.numpy()
        #evtl embed durch die anzahl an Wörtern darin teilen ? beim Finalen auch?
        if(inIteration1 == True):
            bestEmbedding = embed
        elif(angle_between(queryV, bestEmbedding) > angle_between(queryV, embed)): #if a better trining sentence was found
            bestEmbedding = embed
        inIteration1 = False;
            
    return bestEmbedding
#print(getVectorOfAnswer(0))

TypeError: getVectorOfAnswer() missing 1 required positional argument: 'queryV'

In [7]:
import numpy as np
def getVectorOfQuery(query):
    sentence = Sentence(query)
    german_embedding.embed(sentence)
    
    embed = 0
    for token in sentence:
        #print(token)
        print(str(token) + ": ")
        print(np.linalg.norm(token.embedding.numpy()))
        embed += token.embedding.numpy()
    return embed


def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))



In [9]:
def containsWrongWords(query):
    sentence = Sentence(query)
    crawl_embedding.embed(sentence)
    
    for token in sentence:
        print(token)
        print(np.linalg.norm(token.embedding.numpy()))
        if(np.linalg.norm(token.embedding.numpy()) == 0.0):
            return True
    return False
    

In [10]:
def getAnswer(question):
    query = preProcessQuery(question)
    
    pQ = getPlayersInQuery(query)
    lQ = getLeaguesInQuery(pQ[1])
    tQ = getTeamsInQuery(lQ[1])
    sQ = getSeasonsInQuery(tQ[1])
    yQ = getStagesInQuery(sQ[1])
    #print("Query, aus dem der Vektor gemacht wird:")
    print(yQ[1]) # die letzt query die dann auch dem vector gegeben wird
    print("")
    
    #if(containsWrongWords(yQ[1])):
    #    return "Deine Antwort enthält ein oder mehrer unbekannte Wörter!"
    
    queryV = getVectorOfQuery(yQ[1]) # erwarter preprocessed Query, TODO 
    
    queryRefined = [pQ[0],lQ[0],tQ[0],sQ[0],yQ[0]]
    #queryV = getVectorOfQuery(pQ[1]) # erwarter preprocessed Query, TODO 
    #print(getPlayersInQuery(query)[1])

    #queryRefined = [pQ[0], getLeaguesInQuery(query),getTeamsInQuery(query),getSeasonsInQuery(query),getStagesInQuery(query)]
    print("gefundene Dinge: ")
    print(queryRefined)
    answers = sqlAnswers.getAnswers()


    queryToCompare = [len(queryRefined[0]),len(queryRefined[1]),len(queryRefined[2]),len(queryRefined[3]),len(queryRefined[4])]
    possibleAnswers = [] 
    #contains index with possible ansers which could match the query depending
    #on number of player occurances etc

    for x in range(0,len(answers)): #check for every "answer", if [1,0,0,0] matches [1,0,0,0] etc
        sqlMatches = True;
        for i in range(0, len(queryToCompare)): #interate through the whole [0,1,2,3] array and check 
            if(queryToCompare[i] != answers[x][i]):  # if something doesnt match, throw it out
                sqlMatches = False
        if(sqlMatches):
            possibleAnswers.append(x)
        

    print("In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: ")       
    print(possibleAnswers)
    if(len(possibleAnswers) == 1):
        return answers[possibleAnswers[0]][5](queryRefined)
    if(len(possibleAnswers) == 0):
        return "Es konnte leider keine passende Antwort gefunden werden!"
    angles = []
    for answerIndex in possibleAnswers:
        angles.append(angle_between(getVectorOfAnswer(answerIndex, queryV),queryV))

    print(angles)
    print("Beste gefundene Antwort anhand der Cosinus Ähnlichkeit: " + str(angles.index(min(angles))))

    return answers[angles.index(min(angles))][5](queryRefined)

In [11]:
#Stellen Sie Ihre Frage:
eingabe = input("")
print(getAnswer(eingabe))

wie schwer ist Thomas Müller?
wie schwer ist SPIELER

Token: 1 wie: 
8.033962
Token: 2 schwer: 
7.400247
Token: 3 ist: 
6.84859
Token: 4 SPIELER: 
7.6537137
gefundene Dinge: 
[[10243], [], [], [], []]
In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: 
[0, 1, 2, 3]
[0.7266068, 0.5812133, 0.822204, 0.8671051]
Beste gefundene Antwort anhand der Cosinus Ähnlichkeit: 1
Die Person wiegt 74kg.


Manuel Neuer und Thomas Müller 2009 Bundesliga
Wo spielte Thomas Müller Heimverein?
Was ist der Heimverein von Manuel Neuer?
Was ist der Heimverein von Toni Kroos?

Wo hat Manuel Neuer 2008/2009 gespielt?
Wo hat Manuel Neuer 2014/2015 gespielt?

was ist der bevorzugte Fuß von Toni Kroos?

Was ist das Geburtsdatum von Jerome Boateng?

In [ ]:
print(getVectorOfQuery("Verein"))